In [2]:
import numpy as np
import pandas as pd

In [37]:
dict_words = pd.read_csv('Syllable_dictionary.txt', header = None)

In [68]:
syllables = {}
for i in range(len(dict_words)):
    row = dict_words[0][i]
    word = ''
    for j in range(len(row)):
        if(row[j]!=' '):
            word = word+row[j]
        else:
            syllable_list = []
            current_string = ''
            for j1 in range(j+1, len(row)):
                if(row[j1] == ' '):
                    syllable_list.append(current_string)
                    current_string = ''
                else:
                    current_string = current_string + row[j1]
                    if(j1 == len(row) - 1):
                        syllable_list.append(current_string)
                        
            syllable_list_2 = []     
            for s in syllable_list:
                if(len(s) == 1):
                    syllable_list_2.append(int(s))
                else:
                    syllable_list_2.append(int(s[1]))
            syllables[word] = syllable_list_2
            break
            

In [69]:
syllables

{"'gainst": [1],
 "'greeing": [1, 2],
 "'scaped": [1],
 "'tis": [1],
 "'twixt": [1],
 'a': [1],
 'a-doting': [2, 3],
 'abhor': [2],
 'abide': [2],
 'able': [2],
 'about': [2],
 'above': [2],
 'absence': [2],
 'absent': [2],
 'abundance': [3],
 'abundant': [3],
 'abuse': [2],
 'abused': [2],
 'abuses': [3],
 'abysm': [2],
 'accents': [2],
 'acceptable': [4],
 'acceptance': [3],
 'accessary': [4],
 'accident': [3],
 'accidents': [3],
 'account': [2],
 'accumulate': [4],
 'accuse': [2],
 'accusing': [3],
 'achieve': [2],
 'acknowledge': [3],
 'acquaintance': [3],
 'acquainted': [2, 3],
 'act': [1],
 'action': [2],
 'active': [2],
 'actor': [2],
 'add': [1],
 'added': [2],
 "adder's": [2],
 'addeth': [2],
 'adding': [2],
 'addition': [3],
 'adieu': [2],
 'adjunct': [2],
 'admire': [2],
 'admired': [3],
 'admiring': [3],
 'admit': [2],
 'admitted': [3],
 'adonis': [3],
 'adore': [2],
 'adulterate': [3],
 'advance': [2],
 'advantage': [3],
 'adverse': [2],
 'advised': [2],
 'advocate': [3],


In [89]:
raw_data = pd.read_csv('shakespeare.txt',header=None,sep='\n')
raw_data

,0
0,1
1,"From fairest creatures we desire increase,"
2,"That thereby beauty's rose might never die,"
3,"But as the riper should by time decease,"
4,His tender heir might bear his memory:
...,...
2304,"Which from Love's fire took heat perpetual,"
2305,"Growing a bath and healthful remedy,"
2306,"For men discased, but I my mistress' thrall,"
2307,"Came there for cure and this by that I prove,"


In [132]:
int('5')

5

In [139]:
sonnets = []
sonnet = ''
for i in range(1,len(raw_data)):
    if(len(raw_data[0][i].strip()) < 4):
        #i = i + 1
        sonnets.append(sonnet)
        sonnet = ''
    elif(i == len(raw_data) - 1):
        sonnet = sonnet + raw_data[0][i] + '\n'
        sonnets.append(sonnet)
    else:
        sonnet = sonnet + raw_data[0][i] + '\n'
print(len(sonnets))

154


In [140]:
print(sonnets[153])

The little Love-god lying once asleep,
Laid by his side his heart-inflaming brand,
Whilst many nymphs that vowed chaste life to keep,
Came tripping by, but in her maiden hand,
The fairest votary took up that fire,
Which many legions of true hearts had warmed,
And so the general of hot desire,
Was sleeping by a virgin hand disarmed.
This brand she quenched in a cool well by,
Which from Love's fire took heat perpetual,
Growing a bath and healthful remedy,
For men discased, but I my mistress' thrall,
  Came there for cure and this by that I prove,
  Love's fire heats water, water cools not love.

